# ReAct Agent with Multiple Tools

This notebook implements a ReAct (Reasoning and Acting) agent as described by Yao et al. [1], incorporating three main tools: search, compare, and analyze. The agent handles complex queries by reasoning about which tool to use and when.

## Overview

The implementation includes:

1. **Search Tool**: Uses SerpAPI to search the web and format results for the ReAct agent
2. **Compare Tool**: A custom LangChain tool that compares multiple items along a category
3. **Analyze Tool**: Summarizes and extracts key information from search results or comparisons
4. **ReAct Agent Integration**: Uses LangChain's `initialize_agent` with `AgentType.ZERO_SHOT_REACT_DESCRIPTION` to orchestrate the tools
5. **Streamlit UI**: A user interface for interacting with the agent

## References

[1] Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2022). ReAct: Synergizing reasoning and acting in language models. arXiv preprint arXiv:2210.03629.

In [1]:
# # Install required packages
# !pip install langchain_openai google-search-results langchain
# !pip install -U langchain-community

In [2]:
!pip install -q langchain langchain-community langchain-google-genai google-search-results streamlit python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
# === Core setup & imports ===
import os
from dotenv import load_dotenv

load_dotenv()  # loads GOOGLE_API_KEY / SERPAPI_API_KEY from .env if present

# LangChain core (we pinned to a 0.2.x version)
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# LLMs (Gemini via LangChain)
from langchain_google_genai import ChatGoogleGenerativeAI

# SerpAPI utility for search tool
from langchain_community.utilities import SerpAPIWrapper


/Users/zubair/Desktop/Columbia/Assignments/LLM/hw4/q3/.venv-q3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# === API keys ===
# If you have a .env file, these might already be set.
# Otherwise, set them in your environment or .env file.

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY", "YOUR_GEMINI_API_KEY_HERE")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY", "YOUR_SERPAPI_API_KEY_HERE")

# === Main LLM: lighter, fast Gemini model ===
# We choose a "flash" model to keep it cheap and responsive.
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-latest",  # lighter/faster model
    temperature=0.0,              # deterministic for consistent results
)

llm  # quick sanity check: should print a ChatGoogleGenerativeAI object


ChatGoogleGenerativeAI(model='models/gemini-flash-latest', temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x11bd43e10>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x12b462f10>, default_metadata=())

In [14]:
# === Test Gemini via LangChain ===
try:
    response = llm.invoke("Reply with exactly: GEMINI_OK")
    print("Gemini test response:", response)
except Exception as e:
    print("❌ Gemini test failed:")
    print(e)


Gemini test response: content='GEMINI_OK' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-d7838c44-dd22-4181-a0f9-b9bec4aba90d-0' usage_metadata={'input_tokens': 9, 'output_tokens': 4, 'total_tokens': 13}


In [4]:
# === Test SerpAPI ===
try:
    search = SerpAPIWrapper()  # uses SERPAPI_API_KEY from environment
    serp_result = search.run("LangChain ReAct agent example")
    print("SerpAPI test result (first 500 chars):\n")
    print(serp_result[:500])
except Exception as e:
    print("❌ SerpAPI test failed:")
    print(e)


SerpAPI test result (first 500 chars):

['Create an agent that uses ReAct prompting. Based on paper “ReAct: Synergizing Reasoning and Acting in Language Models” (https://arxiv.org/abs/2210.03629)', 'Tool use in the ReAct loop. Agents follow the ReAct (“Reasoning + Acting”) pattern, alternating between brief reasoning steps with targeted tool calls and ...', "In this article, we'll explore the inner workings of React agents, their role in AI development, and how to build them from scratch using Python.", 'Learn about the LangChain ReAc


## Search Tool Implementation

In [ ]:
# Load the search tool using SerpAPI
# search_tool = load_tools(["serpapi"])

In [5]:
# === Search Tool for ReAct agent ===

# Reuse a single SerpAPIWrapper instance
serp = SerpAPIWrapper()

def search_tool(query: str, num_results: int = 5) -> str:
    """
    Use SerpAPI to search the web and return a compact, LLM-friendly summary
    of the top results.

    The ReAct agent will read this text in its 'Observation' step.
    """
    # SerpAPIWrapper has a .results() method that returns structured data
    # (dict with 'organic_results', etc.) in many versions.
    try:
        raw_results = serp.results(query, num_results=num_results)
    except TypeError:
        # Fallback in case this version of SerpAPIWrapper has a different signature
        raw_results = serp.results(query)

    organic = raw_results.get("organic_results", [])
    if not organic:
        # Fallback to the simple .run() text output if structure isn't as expected
        return serp.run(query)

    # Build a clean, numbered list of results
    lines = [f"Search results for: {query}"]
    for i, item in enumerate(organic[:num_results], start=1):
        title = item.get("title") or "No title"
        snippet = item.get("snippet") or item.get("snippet_highlighted_words") or ""
        link = item.get("link") or item.get("url") or ""
        lines.append(f"{i}. {title}\n   Snippet: {snippet}\n   URL: {link}")

    return "\n".join(lines)


# Wrap as a LangChain Tool so the ReAct agent can call it
search_tool_for_agent = Tool(
    name="Search",
    func=search_tool,
    description=(
        "Use this tool to search the web for up-to-date information. "
        "Input should be a natural language search query. "
        "The tool returns a numbered list of results with titles, snippets, and URLs."
    ),
)


In [6]:
test_output = search_tool("ReAct reasoning and acting agents paper", num_results=3)
print(test_output)

Search results for: ReAct reasoning and acting agents paper
1. Synergizing Reasoning and Acting in Language Models
   Snippet: Abstract page for arXiv paper 2210.03629: ReAct: Synergizing Reasoning and Acting in Language Models.
   URL: https://arxiv.org/abs/2210.03629
2. ReAct: Synergizing Reasoning and Acting in Language ...
   Snippet: In this paper, we explore the use of LLMs to generate both reasoning traces and task-specific actions in an interleaved manner, allowing for ...
   URL: https://arxiv.org/pdf/2210.03629
3. ReAct: Synergizing Reasoning and Acting in Language Models
   Snippet: In this paper, we explore the use of LLMs to generate both reasoning traces and task-specific actions in an interleaved manner, allowing for greater synergy ...
   URL: https://react-lm.github.io/


## Comparison Tool Implementation

In [7]:
# === Comparison Tool for ReAct agent ===

def _parse_compare_query(query: str):
    """
    Parse a comparison query of the form:
        'item1, item2, ..., category'

    Returns (items_list, category) or (None, error_message).
    """
    # Split by comma
    parts = [p.strip() for p in query.split(",") if p.strip()]

    # Need at least: item1, item2, category  -> 3 parts
    if len(parts) < 3:
        return None, (
            "Invalid input for Compare tool. "
            "Please provide at least two items and one category, e.g.: "
            "'iPhone 15, Pixel 9, battery life'."
        )

    *items, category = parts

    if len(items) < 2:
        return None, (
            "Invalid input for Compare tool. "
            "Please provide at least two items before the category."
        )

    return (items, category), None


# Prompt template as a normal Python string (could also use PromptTemplate if you like)
COMPARE_PROMPT_TEMPLATE = """
You are an expert comparison assistant.

Your task:
- Compare the following items on the category: "{category}".

Items:
{items_block}

Please:
1. Briefly describe each item (if possible).
2. Compare them specifically on "{category}" (strengths & weaknesses).
3. End with a short, clear recommendation: which item is best for "{category}" and why.

Format:
- Use short headings or bullet points.
- Be concise but informative.
"""


def compare_items(query: str) -> str:
    """
    Compare multiple items on a given category.

    Expected input format:
        "item1, item2, ..., category"

    Example:
        "iPhone 15, Pixel 9, Galaxy S24, battery life"
    """
    parsed, error = _parse_compare_query(query)
    if error:
        # Return the error as a string so the agent can see it and adjust
        return error

    items, category = parsed

    # Build a nice markdown list for the prompt
    items_block = "\n".join(f"- {item}" for item in items)

    prompt = COMPARE_PROMPT_TEMPLATE.format(
        category=category,
        items_block=items_block,
    )

    # Call the LLM (Gemini) to do the actual comparison
    try:
        response = llm.invoke(prompt)
    except Exception as e:
        # Fail gracefully so the agent can recover
        return f"Compare tool failed to generate a comparison. Error: {e}"

    # Chat models usually return an object with .content
    if hasattr(response, "content"):
        return response.content
    else:
        return str(response)


# Wrap this as a LangChain Tool so the ReAct agent can use it
compare_tool_for_agent = Tool(
    name="Compare",
    func=compare_items,
    description=(
        "Use this to compare multiple items along a single category. "
        "Input format: 'item1, item2, ..., category'. "
        "Example: 'iPhone 15, Pixel 9, battery life'. "
        "The tool returns a concise, structured comparison and recommendation."
    ),
)


In [8]:
test_compare = compare_items("iPhone 15, Google Pixel 9, battery life")
print(test_compare)

## Comparison: iPhone 15 vs. Google Pixel 9 (Battery Life)

---

### 1. Item Descriptions

| Item | Description |
| :--- | :--- |
| **iPhone 15** | Apple's standard 6.1-inch flagship (released 2023). Utilizes the highly efficient A16 Bionic chip and optimized iOS software. |
| **Google Pixel 9** | Google's upcoming standard flagship (expected Fall 2024). Will feature the new Tensor G4 chip and focus heavily on AI integration within the Android OS. *(Note: Specifications are based on projections and leaks as the device is unreleased.)* |

---

### 2. Battery Life Comparison

#### iPhone 15

**Strengths:**
*   **Superior Optimization:** iOS and the A16 Bionic chip provide industry-leading efficiency, ensuring consistent, predictable battery drain.
*   **Reliable All-Day Use:** Easily handles a full day of moderate use; rated for up to 20 hours of video playback.
*   **Excellent Standby Time:** Minimal battery drain when the device is idle.

**Weaknesses:**
*   **Smaller Capacity:** Physi

## Analysis Tool Implementation

In [9]:
# === Analysis Tool for ReAct agent ===

ANALYZE_PROMPT_TEMPLATE = """
You are an expert analysis assistant.

Your job is to help a user with the following question:
"{query}"

You are given the following information (which may come from web search results or a comparison of items):

---
{results}
---

Please:
1. Summarize the most important points relevant to the user's question.
2. Highlight any trade-offs, pros/cons, or key insights.
3. If appropriate, make a clear recommendation or conclusion.
4. Keep the answer concise and easy to read (short paragraphs or bullet points).

Do NOT just repeat the text; extract and organize the key information for the user.
"""


def analyze_results(results: str, query: str) -> str:
    """
    Analyze and summarize given results in the context of the user's query.

    - `results`: text from search results, a comparison tool, etc.
    - `query`: the original question the user asked.

    Returns a concise, relevant analysis.
    """
    prompt = ANALYZE_PROMPT_TEMPLATE.format(
        query=query,
        results=results,
    )

    try:
        response = llm.invoke(prompt)
    except Exception as e:
        return f"Analyze tool failed to generate an analysis. Error: {e}"

    if hasattr(response, "content"):
        return response.content
    else:
        return str(response)


def analyze_tool(input_text: str) -> str:
    """
    Wrapper to make the analysis function easy for the ReAct agent to use.

    Expected formats:
    - Just the info to analyze:
        input_text = "some long search or comparison output..."
      (in this case, the query is treated as 'Unknown question').

    - Or include the original question with a delimiter:
        input_text = "What is the best phone for battery life? ||| [info to analyze]"

      i.e. 'query ||| results'
    """
    if "|||" in input_text:
        query, results = input_text.split("|||", 1)
        query = query.strip()
        results = results.strip()
    else:
        query = "Unknown question"
        results = input_text.strip()

    return analyze_results(results=results, query=query)


# Wrap as a LangChain Tool
analyze_tool_for_agent = Tool(
    name="Analyze",
    func=analyze_tool,
    description=(
        "Use this to summarize and extract key information from search results or "
        "comparison outputs. "
        "Input can be either the raw text to analyze, or 'user question ||| text to analyze' "
        "for more context."
    ),
)


In [10]:
test_results = """
1. iPhone 15: Strong performance, good battery life, expensive.
2. Pixel 9: Excellent camera, solid battery, slightly cheaper.
3. Galaxy S24: Very bright screen, decent battery, frequent discounts.
"""

test_analysis = analyze_results(test_results, "Which phone is best if I care most about battery life and price?")
print(test_analysis)


This analysis focuses solely on your two primary criteria: **Battery Life** and **Price**.

---

## 1. Summary of Relevant Points

| Phone | Battery Life (Priority) | Price (Priority) | Key Takeaway |
| :--- | :--- | :--- | :--- |
| **iPhone 15** | Strongest stated battery life ("Good") | Most expensive | Best battery, worst price. |
| **Pixel 9** | Mid-range battery life ("Solid") | Slightly cheaper | Good balance of battery and price. |
| **Galaxy S24** | Weakest stated battery life ("Decent") | Potential for lowest price (Frequent discounts) | Weakest battery, best potential price. |

## 2. Trade-offs and Key Insights

*   **The Battery vs. Price Trade-off:** You must decide whether maximizing battery life is worth the highest cost, or if saving money is worth accepting "Decent" battery performance.
*   **iPhone 15:** Offers the highest stated battery performance ("Good"), but requires the largest initial investment ("Expensive").
*   **Galaxy S24:** While its battery life is descri

## ReAct Agent Integration

In [ ]:
# Integrate tools with ReAct agent

In [ ]:
# def process_query(query: str, max_steps: int = 100) -> str:
#     try:
#         return agent({"input": query, "max_iterations": max_steps})["output"]
#     except RecursionError:
#         return "The query was too complex and exceeded the maximum number of steps. Please try a simpler query."
#     except Exception as e:
#         return f"An error occurred: {str(e)}"

In [32]:
# === Integrate tools with a ReAct agent ===

# 1. Assemble all tools
tools = [
    search_tool_for_agent,
    compare_tool_for_agent,
    analyze_tool_for_agent,
]

# 2. Initialize a ZERO_SHOT_REACT_DESCRIPTION agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,                 # prints Thoughts/Actions/Observations in notebook
    handle_parsing_errors=True,   # lets the agent recover from minor format issues
    max_iterations=5,             # safety cap on ReAct loops
    early_stopping_method="generate",
    return_intermediate_steps=True,  # so we can later inspect the trace
)


# 3. Wrapper function for clean error handling (as in starter code)
def process_query(query: str, max_steps: int = 5) -> str:
    """
    Run the ReAct agent on a user query and return the final answer.

    max_steps: maximum number of reasoning/tool-use steps allowed.
    """
    try:
        # Pass max_iterations per call so you can override the default if you want
        result = agent.invoke({"input": query, "max_iterations": max_steps})
        # result is a dict with at least: {"output": "...", "intermediate_steps": [...]}
        return result["output"]
    except RecursionError:
        return (
            "The query was too complex and exceeded the maximum number of steps. "
            "Please try a simpler query."
        )
    except Exception as e:
        return f"An error occurred while processing your query: {str(e)}"


## Test Your Implementation

Use the cell below to test your implementation with a sample query.

In [17]:
# Test your implementation
sample_query = "What are the top 3 smartphones in 2023, and how do they compare in terms of camera quality and battery life?"

result = process_query(sample_query, max_steps=3)
print(result)



> Entering new AgentExecutor chain...
Question: What are the top 3 smartphones in 2023, and how do they compare in terms of camera quality and battery life?
Thought: I need to identify the top 3 smartphones of 2023 first. Then, I will compare those three phones based on camera quality and battery life.

Action: Search
Action Input: top 3 smartphones of 2023
Observation: Search results for: top 3 smartphones of 2023
1. My Favorite Phones Of 2023: Something For Everyone
   Snippet: My Favorite Phones Of 2023: Something For Everyone · Favorite Overall Phone — Tie: Samsung Galaxy S23 Ultra And iPhone 15 Pro Max · Favorite Phone ...
   URL: https://www.forbes.com/sites/moorinsights/2024/01/23/my-favorite-phones-of-2023-something-for-everyone/
2. Pickr's Best Picks: The best phones of 2023
   Snippet: Best Android phone: Google Pixel 8 Pro · Best iPhone: Apple iPhone 15 Pro Max · Best foldable phone: Motorola Razr 40 Ultra · Best value phone: ...
   URL: https://www.pickr.com.au/buyers-gui

In [35]:
# Test your implementation
sample_query = "What are the top 3 cameras in 2023, and how do they compare in terms of camera quality and battery life?"
print("=================TRACE==================")
result = process_query(sample_query, max_steps=3)
print("=================FINAL ANSWER==================")
print(result)

=================TRACE==================


> Entering new AgentExecutor chain...
Action: Search
Action Input: top 3 cameras of 2023
Observation: Search results for: top 3 cameras of 2023
1. My Top 10 Cameras of 2023!
   Snippet: 10 - Fuji X10 · 9 - Sony A7RV · 8 - Canon EOS-R · 7 - Yashica Mat 124G · 6 - Leica M8 · 5 - Fuji X100V · 4 - Pentax 645Z · 3 - Bronica GS-1.
   URL: https://www.rossjukesphoto.co.uk/photographyblog/my-top-ten-cameras-of-2023
2. Best DSLR 2023
   Snippet: The Canon 1Dx III is impressive for action, the Nikon D850 is arguably one of the best overall, and the Canon 7D Mark II is great for sports.
   URL: https://www.cameralabs.com/best-dslr/
3. The 6 Best Cameras For Photography of 2025
   Snippet: Canon EOS R7: The Canon EOS R7 is an excellent upper mid-range camera that's well-suited to wildlife photography. · Nikon D780: · OM SYSTEM OM-1 ...
   URL: https://www.rtings.com/camera/reviews/best/by-usage/photography
4. The Best Digital Cameras We've Tested for 2025

In [37]:
print(result)

Thought:The user wants the top 3 cameras of 2023 and a comparison of their camera quality and battery life. I have identified the top three (Nikon Z8, Canon EOS R6 Mark II, Panasonic Lumix S5 II) and performed a detailed comparison using the tools. The final analysis provides a clear breakdown of the requested criteria. I will now present the final answer.
Action: Analyze
Action Input: user question ||| ## Expert Comparison: Camera Quality and Battery Life

The following comparison analyzes the Nikon Z8, Canon EOS R6 Mark II, and Panasonic Lumix S5 II based on their performance in camera quality (resolution, sensor performance) and battery life.

---

### 1. Item Descriptions

| Item | Description |
| :--- | :--- |
| **Nikon Z8** | A high-end, professional mirrorless camera featuring a stacked 45.7MP full-frame sensor, designed for speed, resolution, and robust video capabilities (8K). |
| **Canon EOS R6 Mark II** | A versatile, mid-to-high-range mirrorless camera with a 24.2MP full-fr

In [38]:
# Test your implementation
sample_query = "What are the top soccer clubs in 2025, and how do they compare in terms of play style and trophies"
print("=================TRACE==================")
result = process_query(sample_query, max_steps=5)
print("=================FINAL ANSWER==================")
print(result)

=================TRACE==================


> Entering new AgentExecutor chain...
Question: What are the top soccer clubs in 2025, and how do they compare in terms of play style and trophies
Thought: I need to identify the top soccer clubs for the 2024/2025 period and then compare them based on their play style and trophies. Since 2025 is ongoing or just starting, I will look for recent rankings (late 2024/early 2025) to determine the top clubs.

Action: Search
Action Input: top soccer clubs ranking 2024 2025

Observation: Search results for: top soccer clubs ranking 2024 2025

1. World Football / Soccer Clubs Ranking - FootballDatabase
   Snippet: Updated after matches played on 9 November 2025 ; 1. Bayern München · Germany, 2063 ; 2. Arsenal · England, 2022 ; 3. Paris Saint-Germain · France ...
   URL: https://footballdatabase.com/ranking/world/1
2. Club coefficients | UEFA rankings
   Snippet: 1. Real Madrid ; 2. Bayern München ; 3. Inter ; 4. Man City ; 5. Liverpool.
   URL: https:/

## Example Usage

The notebook includes several example queries that demonstrate the agent's ability to reason about which tool to use and how to interpret results. The agent's reasoning traces show a logical flow of thoughts, actions, and observations, culminating in a final answer.

Example trace format:
```
Thought: I need to find information about top smartphones first
Action: Search[top smartphones 2023]
Observation: [Search results about top smartphones]
Thought: Now I should compare the top two options
Action: Compare[iPhone 14 Pro, Samsung Galaxy S23 Ultra, smartphones]
Observation: [Comparison result]
Thought: I should analyze this comparison for the user
Action: Analyze[comparison result]
Observation: [Analysis of the comparison]
Final Answer: [Agent's final response to the user's query]
```

The agent seamlessly switches between tools based on the task at hand, reasoning about which tool to use next and how to interpret the results from each tool.

## References

[1] Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2022). ReAct: Synergizing reasoning and acting in language models. arXiv preprint arXiv:2210.03629. https://arxiv.org/pdf/2210.03629